In [1]:
import sys, os
from stand_dataset import *
import tensorflow as tf
from matplotlib import pyplot

# add root directory
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)

from mrcnn.model import MaskRCNN
from mrcnn.utils import compute_ap
from mrcnn import visualize

ModuleNotFoundError: No module named 'tensorflow'

### Paths

In [ ]:
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
LOG_PATH = os.path.join(ROOT_DIR, "logs")
MODEL_OUTPUT_PATH = os.path.join(LOG_PATH, 'mask_rcnn_stands.h5')

### Datasets

In [ ]:
train_dataset = StandDataset('train')
valid_dataset = StandDataset('valid')
test_dataset = StandDataset('test')

### Training and Validation

In [ ]:
train_dataset.prepare()
valid_dataset.prepare()

config = StandConfig()

model = MaskRCNN(mode='training',
                config=config,
                model_dir=LOG_PATH)

model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=[ "mrcnn_class_logits", 
                             "mrcnn_bbox_fc", 
                             "mrcnn_bbox", 
                             "mrcnn_mask"])
model.train(train_dataset, valid_dataset,
           learning_rate=config.LEARNING_RATE,
           epochs=1,
           layers="heads")

## Testing/Inference

In [ ]:
inference_config = StandConfig()
inference_config.GPU_COUNT = 1
inference_config.IMAGES_PER_GPU = 1

model = MaskRCNN(mode="inference",
                 config=inference_config, 
                 model_dir=LOG_PATH)
model.load_weights(MODEL_OUTPUT_PATH, by_name=True)

### Full-Dataset (mAP with IOU at 0.5)

In [ ]:
test_dataset.prepare()

aps = []
for image in test_dataset.pool:
    results = model.detect([image.load()], verbose=0)
    r = results[0]
    
    masks, class_ids = image.masks()
    calculation = compute_ap(masks, 
                             class_ids, 
                             r['masks'], 
                             r['class_ids'])
    ap = calculation[0]
    aps.append(ap)

mAP = np.mean(aps)
print(f'Model Mean Average Precision (mAP at 0.5): {mAP:.4f}')

### Single Image Test

In [ ]:
image = test_dataset.get_random()
masks, class_ids = image.masks()
loaded_image = image.load()
results = model.detect([loaded_image], verbose=1)
r = results[0]

visualize.display_instances(loaded_image, 
                            r['rois'], 
                            r['masks'], 
                            r['class_ids'],
                            ["BG", "Stand"], 
                            r['scores'])